In [1]:
from pyspark.sql.functions import col, sum, acos, cos, radians, sin
from pyspark.sql.types import FloatType
from pyspark.sql.types import *
import pyspark 
from delta import *
from delta.tables import DeltaTable

def main():

    # Constants
    DISTANCE = 2 # km
    TABLE_PATH = "hdfs:///project/data/business_data/delta_table_proximity_count"
    CSV_PATH = "hdfs:///project/data/business_data/yelp_academic_dataset_business.csv"
    # CSV_PATH = "hdfs:///project/data/business_data/yelp_academic_dataset_business.csv"
    
    # Initiate spark
    builder = pyspark.sql.SparkSession.builder.appName("spark_proximity_count_sql") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.executor.cores", 4)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    
    # Create dataframe from csv file
    schema = "Business_id STRING, Latitude FLOAT, Longitude FLOAT, Stars FLOAT"
    df = spark.read.csv(CSV_PATH, schema=schema).repartition(200, "Business_id")

    # Combination of all business pairs
    df.createOrReplaceTempView("df1")
    df.createOrReplaceTempView("df2")
    df_pairs = spark.sql(
        """
        SELECT 
        df1.Business_id AS Business_id1
        ,df2.Business_id AS Business_id2
        ,df1.Latitude AS Latitude1
        ,df1.Longitude AS Longitude1
        ,df2.Latitude AS Latitude2
        ,df2.Longitude AS Longitude2
        FROM df1, df2
        WHERE df1.Business_id <> df2.Business_id
        AND df1.Business_id < df2.Business_id
        """
    )
    

    # Calculate haversine distances
    df_pairs = df_pairs.withColumn("WithinProximity", 
        ((acos(sin(radians("Latitude1")) * sin(radians("Latitude2")) +
            cos(radians("Latitude1")) * cos(radians("Latitude2")) *
            cos(radians("Longitude2") - radians("Longitude1"))) * 6371
    ) <= DISTANCE).cast("integer"))

    df_pairs.createOrReplaceTempView("df")
    df_proximity = spark.sql(
        """
        SELECT Business_id, SUM(ProximityCount) AS ProximityCount
        FROM
        (
            SELECT df.Business_id1 AS Business_id, SUM(df.WithinProximity) AS ProximityCount
            FROM df
            GROUP BY df.Business_id1
            UNION
            SELECT df.Business_id2 AS Business_id, SUM(df.WithinProximity) AS ProximityCount
            FROM df
            GROUP BY df.Business_id2
        )
        GROUP BY Business_id
        """
    )

    df.createOrReplaceTempView("df")
    df_proximity.createOrReplaceTempView("df_proximity")
    df = spark.sql(
        """
        SELECT df.*, df_proximity.ProximityCount
        FROM df
        INNER JOIN df_proximity
        ON df.Business_id = df_proximity.Business_id
        """
    )

    # If no delta table exists, save and exit
    if not DeltaTable.isDeltaTable(spark, TABLE_PATH):
        df.write.format("delta").save(TABLE_PATH)
        return
        
    # # Upsert delta table
    delta_table = DeltaTable.forPath(spark, TABLE_PATH)
    delta_table.alias("old") \
        .merge(
            df.alias("new"),
            "old.Business_id = new.Business_id"
        ) \
        .whenMatchedUpdate(set=
            {
                "Business_id": "new.Business_id",
                "Latitude": "new.Latitude",
                "Longitude": "new.Longitude",
                "Stars": "new.Stars",
                "ProximityCount": "new.ProximityCount"
            }
        ) \
        .whenNotMatchedInsert(values=
            {
                "Business_id": "new.Business_id",
                "Latitude": "new.Latitude",
                "Longitude": "new.Longitude",
                "Stars": "new.Stars",
                "ProximityCount": "new.ProximityCount"
            }
        ) \
        .execute()


if __name__ == "__main__":
    main()

:: loading settings :: url = jar:file:/home/ubuntu/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2aaf48a7-e306-42d4-b673-7013bb03f718;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 232ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

23/04/26 20:46:34 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/04/26 20:46:50 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar added multiple times to distributed cache.
23/04/26 20:46:50 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-storage-2.3.0.jar added multiple times to distributed cache.
23/04/26 20:46:50 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar added multiple times to distributed cache.


ERROR:root:Exception while sending command.age 5:>              (0 + 0) / 40000]
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=65>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart

Py4JError: An error occurred while calling o105.execute